In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%%capture
!pip install ftfy translate langdetect imbalanced-learn collections-extended rake_nltk googletrans

In [3]:
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks')

# Step 1: Importing Dependencies

In [4]:
# !pip install chart_studio
# !pip install wordcloud
# !pip install ftfy
# !pip install rake_nltk
# !pip install gensim 
# !pip install missingno
# !pip install googletrans
# !pip install langdetect
# !pip install imblearn
# !pip install collections2

# Standard Library
import pandas as pd
import numpy as np

from ftfy import *
import nltk
from rake_nltk import Rake

import missingno as msno
import matplotlib.pyplot as plt 
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns 

# For Translation
from googletrans import Translator
from langdetect import detect

#For Upsampling
import imblearn
from collections import Counter
from matplotlib import pyplot
from numpy import where
# from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

# Import label encoder 
from sklearn import preprocessing 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Import Dependencies

import os
import re

import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import keras

from keras import layers
# For Embedding Layer
from keras.layers.embeddings import Embedding

# For RNN Layer
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, Bidirectional
from keras.layers import Dense, Dropout, Activation, Input,  Flatten

# import numpy as np
# from tensorflow import keras
# from tensorflow.keras import layers

# For CNN Layer
from keras.layers import Conv1D, MaxPooling1D

# For Text Summarization
from gensim.summarization import summarize

%tensorflow_version 1.x

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


TensorFlow is already loaded. Please restart the runtime to change versions.


# Step 2 : Importing preprocessed Dataset generated in ML Notebook

In [5]:
filename= "/content/drive/My Drive/Colab Notebooks/final_dataframe_forDeep_Learning_Model.csv"
df = pd.read_csv(filename)

# Step 3 : Label Encoding of Target variable

In [6]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['Assignment group'] = label_encoder.fit_transform(df['Assignment group']) 
  
df['Assignment group'].unique() 

array([ 8, 30, 41, 52, 63, 74, 79, 80,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36,
       37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 64, 65, 33, 66, 67, 68, 69, 70, 71, 72,
       73, 75, 76, 77, 78,  2,  3,  0,  1,  4,  6,  5,  7])

# Step 4 :Tokenizing and Padding the Independent features ('Complete Description')

In [7]:
MAX_LENGTH = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.Complete_Description.values)
post_seq = tokenizer.texts_to_sequences(df.Complete_Description.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [8]:
y = df['Assignment group']

# Step 5 :Splitting the data into Train and Test (Ratio = 0.30)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.3, random_state=1)

In [10]:
X_train.shape,X_test.shape

((36560, 300), (15669, 300))

In [11]:
y_train.shape, y_test.shape

((36560,), (15669,))

# Step 6 :Creating input dimension ('vocab_size')

In [12]:
vocab_size = len(tokenizer.word_index) + 1
max_features = vocab_size 
vocab_size

16390

# Step 7 :Creating Output dimension ('num_class')

In [13]:
num_class = len(np.unique(df['Assignment group'].values))
num_class

81

# Step 8 :Creating sequential Model with LSTM

In [ ]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(MAX_LENGTH, ), dtype="int32")

# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Dropout(0.2)(x)

# Add 2 LSTM layers
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64, return_sequences=False)(x)

x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(num_class, activation='softmax')(x)

# Add a classifier
lstm_model = keras.Model(inputs, outputs)
lstm_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 128)          2097920   
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 300, 64)           49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080

In [ ]:
lstm_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
# lstm_model.fit([X_train], batch_size=20, y=(y_train), verbose=1, validation_split=0.30, epochs=7)
lstm_model.fit(X_train, y_train, batch_size=20, epochs=7, validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 36560 samples, validate on 15669 samples
Epoch 1/7
36560/36560 [==============================] - 1551s 42ms/step - loss: 1.9082 - acc: 0.5113 - val_loss: 0.8525 - val_acc: 0.7704
Epoch 2/7
36560/36560 [==============================] - 1556s 43ms/step - loss: 0.6161 - acc: 0.8363 - val_loss: 0.5228 - val_acc: 0.8673
Epoch 3/7
36560/36560 [==============================] - 1549s 42ms/step - loss: 0.3811 - acc: 0.8970 - val_loss: 0.3482 - val_acc: 0.9142
Epoch 4/7
36560/36560 [==============================] - 1534s 42ms/step - loss: 0.2723 - acc: 0.9261 - val_loss: 0.3143 - val_acc: 0.9223
Epoch 5/7
36560/36560 [==============================] - 1542s 42ms/step - loss: 0.2103 - acc: 0.9418 - val_loss: 0.2820 - val_acc: 0.9350
Epoch 6/7
36560/36560 [==============================] - 1526s 42ms/step - loss: 0.1680 - acc: 0.9549 - val_loss: 0.2638 - val_acc: 0.9408
Epoch 7/7
36560/36560 [==============================] - 1519s 42ms/step - loss: 0.1369 - acc: 0.9626 - val_loss: 0.

In [ ]:
lstm_predicted = lstm_model.predict(X_test)
lstm_predicted = np.argmax(lstm_predicted, axis=1)
accuracy_score(y_test, lstm_predicted)

0.947412087561427

# Step 9 : Implementing GRU 

In [ ]:
max_features = vocab_size  
maxlen = MAX_LENGTH
seq_output_size = 128
filters = 64
kernel_size = 5
pool_size = 4
activation_func = 'sigmoid'

In [ ]:
# GRU 
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
# x = layers.Add(Sequential())(inputs)
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(x)
x = layers.MaxPooling1D(pool_size=pool_size)(x)
# Add GRU
x = layers.GRU(seq_output_size, dropout=0.2, recurrent_dropout=0.2)(x)
x = layers.Dense(1)(x)
# outputs = layers.Activation('sigmoid')(x)
outputs = layers.Dense(num_class, activation='softmax')(x)
# Add a classifier
gru_model = keras.Model(inputs, outputs)
gru_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         2097920   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          41024     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               74112     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129 

In [ ]:
gru_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
gru_model.fit(X_train, y_train, batch_size=20, epochs=7, validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 36560 samples, validate on 15669 samples
Epoch 1/7
36560/36560 [==============================] - 286s 8ms/step - loss: 3.8434 - acc: 0.0307 - val_loss: 3.5350 - val_acc: 0.0476
Epoch 2/7
36560/36560 [==============================] - 278s 8ms/step - loss: 3.4778 - acc: 0.0616 - val_loss: 3.3870 - val_acc: 0.0548
Epoch 3/7
36560/36560 [==============================] - 276s 8ms/step - loss: 3.2342 - acc: 0.0869 - val_loss: 3.2651 - val_acc: 0.0863
Epoch 4/7
36560/36560 [==============================] - 275s 8ms/step - loss: 3.1039 - acc: 0.1171 - val_loss: 3.1823 - val_acc: 0.1134
Epoch 5/7
36560/36560 [==============================] - 273s 7ms/step - loss: 222.9329 - acc: 0.1228 - val_loss: 3.2095 - val_acc: 0.1025
Epoch 6/7
36560/36560 [==============================] - 272s 7ms/step - loss: 3.0228 - acc: 0.1286 - val_loss: 3.1199 - val_acc: 0.1472
Epoch 7/7
36560/36560 [==============================] - 272s 7ms/step - loss: 2.9322 - acc: 0.1475 - val_loss: 3.0749 - val_a

In [ ]:
gru_predicted = gru_model.predict(X_test)
gru_predicted = np.argmax(gru_predicted, axis=1)
accuracy_score(y_test, gru_predicted)

0.13734124704831197

# Step 10 : Implementing Bi LSTM

In [ ]:
max_features = vocab_size  
maxlen = 200
seq_output_size = 128
filters = 64
kernel_size = 5
pool_size = 4
activation_func = 'sigmoid'

In [ ]:
# Bi directional LSTM
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Dropout(0.2)(x)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(num_class, activation='softmax')(x)
# Add a classifier
bilstm_model = keras.Model(inputs, outputs)
bilstm_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 128)         2097920   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                4128

In [ ]:
#model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
bilstm_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
bilstm_model.fit(X_train, y_train, batch_size=20, epochs=7, validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 36560 samples, validate on 15669 samples
Epoch 1/7
36560/36560 [==============================] - 2887s 79ms/step - loss: 1.7656 - acc: 0.5609 - val_loss: 0.6578 - val_acc: 0.8306
Epoch 2/7
36560/36560 [==============================] - 2846s 78ms/step - loss: 0.5187 - acc: 0.8636 - val_loss: 0.4106 - val_acc: 0.8863
Epoch 3/7
36560/36560 [==============================] - 2839s 78ms/step - loss: 0.3040 - acc: 0.9187 - val_loss: 0.3001 - val_acc: 0.9267
Epoch 4/7
36560/36560 [==============================] - 2839s 78ms/step - loss: 0.2291 - acc: 0.9383 - val_loss: 0.2984 - val_acc: 0.9261
Epoch 5/7
36560/36560 [==============================] - 2834s 78ms/step - loss: 0.1703 - acc: 0.9539 - val_loss: 0.2551 - val_acc: 0.9423
Epoch 6/7
36560/36560 [==============================] - 2828s 77ms/step - loss: 0.1357 - acc: 0.9637 - val_loss: 0.2698 - val_acc: 0.9387
Epoch 7/7
36560/36560 [==============================] - 2823s 77ms/step - loss: 0.1203 - acc: 0.9673 - val_loss: 0.